In [ ]:
import os
import random
import shutil
import cv2
import glob
import xml.etree.ElementTree as ET
import csv
import torchvision.transforms as transforms
import torch
import torch.nn as nn
import torchvision.models as models

from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm
from PIL import Image

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data_root = '/content/drive/MyDrive/ai기반 ocr/컴퓨터비전/이상행동감지'
file_root = f'{data_root}/data'

In [ ]:
file_list = os.listdir(f'{file_root}/image_id')
file_list

['C_3_12_40_BU_SMC_10-14_11-43-44_CB_RGB_DF2_M2.mp4',
 'C_3_12_40_BU_SMC_10-14_11-43-44_CA_RGB_DF2_M2.mp4',
 'C_3_12_40_BU_DYB_10-16_14-49-01_CD_RGB_DF2_M1.mp4',
 'C_3_12_40_BU_DYB_10-16_14-49-01_CB_RGB_DF2_M1.mp4',
 'C_3_12_40_BU_SMC_10-14_11-43-44_CC_RGB_DF2_M2.mp4',
 'C_3_12_40_BU_DYA_07-29_16-07-39_CE_RGB_DF2_F2.mp4',
 'C_3_12_40_BU_DYA_07-29_16-07-39_CD_RGB_DF2_F2.mp4',
 'C_3_12_39_BU_SMC_10-14_11-41-49_CE_RGB_DF2_M2.mp4',
 'C_3_12_40_BU_DYB_10-16_14-49-01_CA_RGB_DF2_M1.mp4',
 'C_3_12_40_BU_DYA_07-29_16-07-39_CF_RGB_DF2_F2.mp4']

In [ ]:
# mp4에서 이미지 추출하기
# 각 mp4 이름으로 디렉토리 생성 후 저장

# for file in tqdm(file_list):
#   name = file.split('.')[0]
#   if not os.path.isdir(f'{file_root}/images/{name}'):
#     os.mkdir(f'{file_root}/images/{name}')
#   video_path = f'{file_root}/image_id/{file}'
#   print(video_path)
#   cap = cv2.VideoCapture(video_path)
#   num = 1
#   while(cap.isOpened()):
#     ret, frame = cap.read()
#     if ret:
#       cv2.imwrite(f'{file_root}/images/{name}/{name}_{num:06d}.png', frame)
#       num += 1
#     else:
#       break
#   cap.release()

In [ ]:
# 프레임이 있는 파일 리스트만 추출
file_list = os.listdir(f'{file_root}/image_id')
# len(file_list)
real_file_list = []
for file in file_list:
  temp = os.listdir(f"{file_root}/images/{file.split('.')[-2]}")
  if len(temp) > 0:
    real_file_list.append(file)
len(real_file_list)

10

In [ ]:
real_file_list

['C_3_12_40_BU_SMC_10-14_11-43-44_CB_RGB_DF2_M2.mp4',
 'C_3_12_40_BU_SMC_10-14_11-43-44_CA_RGB_DF2_M2.mp4',
 'C_3_12_40_BU_DYB_10-16_14-49-01_CD_RGB_DF2_M1.mp4',
 'C_3_12_40_BU_DYB_10-16_14-49-01_CB_RGB_DF2_M1.mp4',
 'C_3_12_40_BU_SMC_10-14_11-43-44_CC_RGB_DF2_M2.mp4',
 'C_3_12_40_BU_DYA_07-29_16-07-39_CE_RGB_DF2_F2.mp4',
 'C_3_12_40_BU_DYA_07-29_16-07-39_CD_RGB_DF2_F2.mp4',
 'C_3_12_39_BU_SMC_10-14_11-41-49_CE_RGB_DF2_M2.mp4',
 'C_3_12_40_BU_DYB_10-16_14-49-01_CA_RGB_DF2_M1.mp4',
 'C_3_12_40_BU_DYA_07-29_16-07-39_CF_RGB_DF2_F2.mp4']

In [ ]:
# random.seed(2024)
# random.shuffle(real_file_list)
# test_ratio = 0.1

# num_file = len(real_file_list)

# test_list = real_file_list[:int(num_file*test_ratio)]
# valid_list = real_file_list[int(num_file*test_ratio):int(num_file*test_ratio)*2]
# train_list = real_file_list[int(num_file*test_ratio)*2:]

In [ ]:
def read_one_xml(xml_path):
    tree = ET.parse(xml_path)
    root = tree.getroot()
    event_dict = {
        'event': None,
        'start_frame': -1,
        'end_frame': -1
    }
    for child in root.iter('track'):
        if child.attrib['label'] == 'theft_start':
            event_dict['event'] = child.attrib['label'].split('_')[0]
            frame = child.find('box').attrib['frame']
            event_dict['start_frame'] = int(frame)
        elif child.attrib['label'] == 'theft_end':
            event_dict['event'] = child.attrib['label'].split('_')[0]
            try:
                frame = child.find('box').attrib['frame']
            except:
                return {}, {}
            event_dict['end_frame'] = int(frame)
    return event_dict

In [ ]:
# XML 파일에서 행동 정보 추출하기
event_list = []
# {'event': 'theft', 'start_frame': X, 'end_frame': X}
for i in range(len(real_file_list)):
  file = real_file_list[i]
  xml_path = f"{file_root}/labels_id/{file.replace('mp4', 'xml')}"
  event_dict = read_one_xml(xml_path)
  event_list.append(event_dict)

In [ ]:
event_list

[{'event': 'theft', 'start_frame': 79, 'end_frame': 150},
 {'event': 'theft', 'start_frame': 79, 'end_frame': 151},
 {'event': 'theft', 'start_frame': 80, 'end_frame': 123},
 {'event': 'theft', 'start_frame': 81, 'end_frame': 126},
 {'event': 'theft', 'start_frame': 79, 'end_frame': 150},
 {'event': 'theft', 'start_frame': 121, 'end_frame': 145},
 {'event': 'theft', 'start_frame': 121, 'end_frame': 142},
 {'event': 'theft', 'start_frame': 82, 'end_frame': 145},
 {'event': 'theft', 'start_frame': 81, 'end_frame': 126},
 {'event': 'theft', 'start_frame': 123, 'end_frame': 148}]

In [ ]:
def load_img(link):
  # 1. 이미지 읽어들이기
  image = Image.open(link)

  # 2. 이미지 전처리 및 텐서로 변환
  transform = transforms.Compose([
      transforms.Resize((224, 224)),  # 이미지 크기 조정 (ResNet50이 요구하는 입력 크기)
      transforms.ToTensor(),          # 이미지를 텐서로 변환 (픽셀 값을 [0, 1] 범위로 정규화)
      transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # 이미지 정규화
  ])

  # 3. 변환 적용
  tensor_image = transform(image)

  # 4. 배치 차원 추가 (모델 입력을 위해 배치 차원이 필요함)
  # tensor_image = tensor_image.unsqueeze(0)  # (C, H, W) -> (1, C, H, W)
  # print(tensor_image.shape)  # 텐서 형태 확인
  return tensor_image

In [ ]:
frame_list = []
label_list = []
for i in range(len(real_file_list)-4):
  file = real_file_list[i]
  name = file.split('.')[0]
  if event_list[i] is not None:
      current_frame = 0
      while(current_frame < event_list[i]['start_frame']+4):
          if (current_frame+3) >= event_list[i]['start_frame']:
              event_num = 1
          else:
              event_num = 0
          data = [load_img(f'/content/drive/MyDrive/ai기반 ocr/컴퓨터비전/이상행동감지/data/images/{name}/{name}_{(current_frame+1):06d}.png'),
              load_img(f'/content/drive/MyDrive/ai기반 ocr/컴퓨터비전/이상행동감지/data/images/{name}/{name}_{(current_frame+2):06d}.png'),
              load_img(f'/content/drive/MyDrive/ai기반 ocr/컴퓨터비전/이상행동감지/data/images/{name}/{name}_{(current_frame+3):06d}.png')]
          data = torch.stack(data)
          frame_list.append(data)
          label_list.append(event_num)
          current_frame += 1
      data = [load_img(f'/content/drive/MyDrive/ai기반 ocr/컴퓨터비전/이상행동감지/data/images/{name}/{name}_{(current_frame+1):06d}.png'),
              load_img(f'/content/drive/MyDrive/ai기반 ocr/컴퓨터비전/이상행동감지/data/images/{name}/{name}_{(current_frame+2):06d}.png'),
              load_img(f'/content/drive/MyDrive/ai기반 ocr/컴퓨터비전/이상행동감지/data/images/{name}/{name}_{(current_frame+3):06d}.png')]
      data = torch.stack(data)
      frame_list.append(data)
      label_list.append(event_num)

In [ ]:
import pandas as pd

df = pd.DataFrame({
    'Column1': frame_list,
    'Column2': label_list
})

# 3. 데이터 프레임을 CSV 파일로 저장
csv_file_path = './train.csv'  # 저장할 파일 경로와 이름
df.to_csv(csv_file_path, index=False)

print(f"DataFrame has been saved to {csv_file_path}")

DataFrame has been saved to ./train.csv


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
df_label_1 = df[df['Column2'] == 1]
df_label_0 = df[df['Column2'] == 0].sample(n=len(df_label_1), random_state=42)
df_balanced = pd.concat([df_label_1, df_label_0])
df_balanced = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

In [ ]:
csv_file_path = './diff.csv'  # 저장할 파일 경로와 이름
df_balanced.to_csv(csv_file_path, index=False)

In [ ]:
# frame_list = df['Column1'].to_list()
# label_list = df['Column2'].to_list()

frame_list = df_balanced['Column1'].to_list()
label_list = df_balanced['Column2'].to_list()

In [ ]:
batch_size = 1
data = torch.stack(frame_list)
label = torch.tensor(label_list)
dataset = TensorDataset(data, label)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [ ]:
import torch.nn as nn
import torch
import torch.nn.functional as F
from torchvision import models

class ResNetConv3DLSTM(nn.Module):
    def __init__(self, num_classes=1):  # 이진 분류이므로 num_classes=1
        super(ResNetConv3DLSTM, self).__init__()

        # ResNet50 backbone
        resnet = models.resnet50(pretrained=True)
        self.resnet = nn.Sequential(*list(resnet.children())[:-2])  # Fully connected layer 제거
        self.resnet_out_channels = 2048

        # Conv3D layer
        self.conv3d = nn.Conv3d(in_channels=self.resnet_out_channels, out_channels=512, kernel_size=(3, 3, 3), stride=1, padding=1)

        # LSTM layer
        self.lstm = nn.LSTM(input_size=512, hidden_size=256, num_layers=2, batch_first=True)

        # Fully connected layer
        self.fc = nn.Linear(256, num_classes)

    def forward(self, x):
        batch_size, num_frames, C, H, W = x.shape

        # ResNet feature extraction
        x = x.view(batch_size * num_frames, C, H, W)  # ResNet 입력 준비
        x = self.resnet(x)  # ResNet 출력: [batch_size * num_frames, 2048, H', W']
        x = x.view(batch_size, num_frames, self.resnet_out_channels, x.size(2), x.size(3))  # [batch_size, num_frames, 2048, H', W']
        x = x.permute(0, 2, 1, 3, 4)  # Conv3D 입력: [batch_size, 2048, num_frames, H', W']

        # Conv3D
        x = self.conv3d(x)  # Conv3D 출력: [batch_size, 512, num_frames, H', W']
        x = torch.mean(x, dim=[3, 4])  # Global average pooling: [batch_size, 512, num_frames]

        # LSTM
        x = x.permute(0, 2, 1)  # LSTM 입력: [batch_size, num_frames, 512]
        x, _ = self.lstm(x)  # LSTM 출력: [batch_size, num_frames, 256]
        x = x[:, -1, :]  # 마지막 타임스텝의 hidden state: [batch_size, 256]

        # Fully connected
        x = self.fc(x)  # 출력: [batch_size, 1]

        # Sigmoid activation 추가
        x = torch.sigmoid(x)  # 출력값을 [0, 1] 범위로 변환

        return x.squeeze(1)  # [batch_size]로 크기 맞추기 위해 squeeze

model = ResNetConv3DLSTM(num_classes=1)  # 이진 분류이므로 num_classes=1
model = model.to(device)
criterion = nn.BCELoss()  # 이진 분류를 위한 손실 함수

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 113MB/s]


In [ ]:
import torch

# Initialize optimizer and loss function
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
criterion = torch.nn.BCELoss()  # Assume binary classification

# 예제 학습 루프
num_epochs = 100
best_accuracy = 0.0
best_model_weights = None

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_predictions = 0
    total_samples = 0

    for inputs, labels in dataloader:  # Assume dataloader is defined
        inputs_, labels_ = inputs.to(device), labels.to(device)
        labels_ = labels_.float()
        optimizer.zero_grad()
        outputs = model(inputs_)
        loss = criterion(outputs, labels_)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        # 정확도 계산
        predicted_labels = (outputs >= 0.5).float()  # 0.5를 기준으로 이진 레이블로 변환
        correct_predictions += (predicted_labels == labels_).sum().item()
        total_samples += labels_.size(0)

    # Epoch당 평균 손실과 정확도 계산
    epoch_loss = running_loss / len(dataloader)
    epoch_accuracy = correct_predictions / total_samples

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.8f}, Accuracy: {epoch_accuracy:.8f}")

    # 최고 정확도일 경우 모델 가중치 저장
    if epoch_accuracy > best_accuracy:
        best_accuracy = epoch_accuracy
        best_model_weights = model.state_dict()

# 최종적으로 최고 정확도를 기록한 모델 가중치 저장
if best_model_weights is not None:
    torch.save(best_model_weights, 'best_model_weights2.pth')
    print(f"Best model saved with accuracy: {best_accuracy:.8f}")